In [3]:
import api.openligadb
from api import openligadb

def extract_result(match_results):
    for match_result in match_results:
        if match_result['resultName'] == 'Endergebnis':
            return match_result['pointsTeam1'], match_result['pointsTeam2']
    return None


def extract_winner(match_data):
    if match_data["goalsHome"] > match_data["goalsAway"]:
        return match_data["teamHomeId"]
    elif match_data["goalsHome"] < match_data["goalsAway"]:
        return match_data["teamAwayId"]
    else:
        return None


def extract_match_data(json):
    match_data = {}
    match_data["id"] = json["matchID"]
    match_data["teamHomeId"] = json["team1"]["teamId"]
    match_data["teamAwayId"] = json["team2"]["teamId"]
    match_data["goalsHome"], match_data["goalsAway"] = extract_result(json["matchResults"])
    match_data["winnerTeamId"] = extract_winner(match_data)
    return match_data

In [7]:
json = api.openligadb.get_all_season_matches("bl1", 2023)
match_data = extract_match_data(json[0])
match_data

{'id': 66630,
 'teamHomeId': 134,
 'teamAwayId': 40,
 'goalsHome': 0,
 'goalsAway': 4,
 'winnerTeamId': 40}